In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec.load('Word2vec.model')

In [ ]:
model.wv.vocab

In [ ]:
model.wv[model.wv.index2word[0]].shape[0]

In [1]:
import tensorflow as tf

_buffer_size = 20000
_bucket_size = 1
_thread_num = 16


def get_vocab(vocab_path, isTF=True):
    if isTF:
        vocab_path_tensor = tf.constant(vocab_path)
        tf.add_to_collection(tf.GraphKeys.ASSET_FILEPATHS, vocab_path_tensor)
        vocab_dict = tf.contrib.lookup.index_table_from_file(
            vocabulary_file=vocab_path_tensor,
            num_oov_buckets=0,
            default_value=1)
    else:
        vocab_dict = {}
        with open(vocab_path, "r") as f:
            for vocab in f:
                vocab_dict[len(vocab_dict)] = vocab.strip()
    return vocab_dict

In [2]:
corpus_path = "data/ratings_train.cleaned.txt"
vocab_path = "Word2vec.vocab"
max_len = 100
batch_size = 1

In [3]:
tf_vocab = get_vocab(vocab_path)
dataset = tf.data.TextLineDataset(corpus_path)

# dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(_buffer_size))

dataset = dataset.map(lambda x: tf.string_split([x], delimiter="\t").values)

dataset = dataset.map(lambda x: {
                        "input": tf.string_split([x[0]]).values,
                        "label": tf.string_split([x[1]]).values
                    },
                    num_parallel_calls=_thread_num
                    )

# Truncate to max_len
dataset = dataset.map(lambda x:
            tf.cond(tf.greater(tf.shape(x["input"])[0], max_len),
                    lambda: {
                        "input": x["input"][:max_len],
                        "label": x["label"]
                    },
                    lambda: {
                        "input": x["input"],
                        "label": x["label"]
                    }
                )
            )

dataset = dataset.map(lambda x: {
                        "input": tf_vocab.lookup(x["input"]),
                        "label": x["label"]
                    },
                    num_parallel_calls=_thread_num
                    )

dataset = dataset.map(lambda x: {
                        "input": tf.to_int32(x["input"]),
                        "len": tf.shape(x["input"])[0],
                        "label": tf.strings.to_number(x["label"][0], tf.int32)
                    },
                        num_parallel_calls=_thread_num
                    )

dataset = dataset.padded_batch(
  batch_size,
  {
      "input": [tf.Dimension(max_len)],
      "len": [],
      "label": []
  },
  {
      "input": 0,
      "len": 0,
      "label": 0
  }
)

dataset = dataset.prefetch(3)

In [8]:
iters = tf.data.Iterator.from_structure(dataset.output_types,
                                        dataset.output_shapes)
features = iters.get_next()


# create the initialisation operations
init_op = iters.make_initializer(dataset)

In [9]:
sess = tf.Session()

In [10]:
sess.run(init_op)
sess.run(tf.tables_initializer())

In [11]:
for i in range(2000):
    print(i)
    t = sess.run(features)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

InvalidArgumentError: slice index 1 of dimension 0 out of bounds.
	 [[{{node strided_slice_1}} = StridedSlice[Index=DT_INT32, T=DT_STRING, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1](arg0, strided_slice/stack_1, strided_slice_1/stack_1, strided_slice/stack_1)]]
	 [[node IteratorGetNext_1 (defined at <ipython-input-8-0316891af9f9>:3)  = IteratorGetNext[output_shapes=[[?,100], [?], [?]], output_types=[DT_INT32, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](IteratorV2_1)]]

In [ ]:
t

In [ ]:
tf.strings.to_number

In [ ]:
tf.cast(tf.constant("good", dtype=tf.string), tf.int32)

In [ ]:
a = tf.constant([[1,2,3,4],[2,4,5,9]])

In [ ]:
tf.sum(a, axis=1)

In [ ]:
x = tf.placeholder(shape=(None,128), dtype=tf.float32)

In [ ]:
tf.layers.dense(x, x.shape[1]//2)